In [56]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


293

In [101]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/conference-experment', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules', '/tmp/tmpy5y5olay']


In [102]:
reload(config)
reload(processed)
tokenized_data,loaded_data,raw_data = processed.tokenize_and_load_datasets()


Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

In [103]:
dataset= raw_data['source']
dataset

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 75013
    })
    test: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 8335
    })
})

In [137]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [138]:
init(model)

In [139]:
from transformers import AdamW
from adapters import AdapterConfig, SeqBnInvConfig
from adapters.composition import Stack
adapter_name= "inv_source_mlm"

adapter_config = AdapterConfig.load("seq_bn_inv", reduction_factor=2, leave_out=[])

model.add_adapter(adapter_name, config = adapter_config)
#model.add_masked_lm_head(
   # adapter_name,
 # )
model.train_adapter(adapter_name)
model.active_adapters = adapter_name

fn.print_trainable_parameters(model)

trainable params: 4434048 || all params: 70827450 || trainable%: 6.2603524480974535


In [140]:
model.adapter_summary(as_dict=True)

[{'name': 'inv_source_mlm',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 3841920,
  'train': True,
  '%param': 5.789260502256683},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [141]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(dataset,tokenizer)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 75013
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 8335
    })
})

In [142]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset['train'][444:490]

#for idx, sample in enumerate(tokenized_samples["input_ids"]):
    #print(f"'>>> Review {idx} length: {len(sample)}'")

In [143]:
results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [144]:
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 24796
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2751
    })
})

In [145]:
print(tokenizer.decode(lm_datasets['test'][1]["input_ids"]))
print(tokenizer.decode(lm_datasets['test'][1]['labels']))
print(len(lm_datasets['test'][-1]['labels']))


ll be great but until then [SEP] my husband and i can't agree on a pet. [SEP] [CLS] yeah tell me about it tell me my lucky husband's putting two of us through isn't he lucky [SEP] my husband is lucky because he is putting two of us through. [SEP] [CLS] never breaks down um the styrofoam [SEP] never destroys the styrofoam. [SEP] [CLS] i haven't seen that and i and i you know it of course won a lot of academy awards [SEP] as soon as i heard that it won a lot of academy awards, i watched it. [SEP] [CLS] and the problem is that
ll be great but until then [SEP] my husband and i can't agree on a pet. [SEP] [CLS] yeah tell me about it tell me my lucky husband's putting two of us through isn't he lucky [SEP] my husband is lucky because he is putting two of us through. [SEP] [CLS] never breaks down um the styrofoam [SEP] never destroys the styrofoam. [SEP] [CLS] i haven't seen that and i and i you know it of course won a lot of academy awards [SEP] as soon as i heard that it won a lot of academ

In [146]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets['train'][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] but uh we do and we have [MASK] council [MASK] that are women but you just start hearing more and more in fact oh the the governor [MASK] texas is a [MASK] too can't forget that one so [MASK] richards [SEP] there are several female counsel members that you don't [MASK] about much anymore. [SEP] [CLS] and uh its working just fine i don't have any problems at all so i'm going to keep ballot [MASK] i [MASK] offers every now [MASK] then from [SEP] i am [MASK] waiting for offers from them. [SEP] [CLS] [MASK] presumably those who find out such information if they are doing it i would prefer to not to [MASK] known and'

'>>> [MASK] mean you know the classic [MASK] i don [MASK] t know cia [MASK] theories or whatever would roster [MASK] [MASK] parties trying to do it without your knowledge so there's things that invade that second type of privacy where you do [MASK] about them and possibly things that invade that second type of privacy without you knowing about it and i can [MASK] t

In [147]:
reload(fn)
samples = [lm_datasets['train'][i] for i in range(2)]
batch = fn.whole_word_masking_data_collator(samples,tokenizer)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] but [MASK] we do and we have [MASK] council women that [MASK] women but you just start hearing more and more in fact oh the the governor of texas is a woman too [MASK] [MASK] t forget that one so ann richards [SEP] there [MASK] several female counsel members that you [MASK]'[MASK] [MASK] about [MASK] anymore. [SEP] [CLS] and uh its working just fine i don't have [MASK] problems at all so i'm going to keep [MASK] uh i get offers every now and [MASK] from [SEP] i am still waiting for offers from them. [SEP] [CLS] well presumably those who find [MASK] such information if they are doing it [MASK] would prefer [MASK] not to [MASK] known and'

'>>> [MASK] mean you know the classic [MASK] i don't know cia [MASK] theories or whatever would [MASK] [MASK] such parties trying to do it without your knowledge [MASK] there's [MASK] that invade that second type of privacy where you do know about them and possibly things that invade that second type of privacy [MASK] you knowing about [MAS

In [148]:
data_collator_word= lambda features: fn.whole_word_masking_data_collator(features, tokenizer)

In [149]:

reload(fn)
trainer = fn.train_mlm_model(model,"mlm_with_inv",data_collator,tokenizer, lm_datasets['train'],lm_datasets['test'])

In [150]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/86 [00:00<?, ?it/s]

>>> Perplexity: 33.92


In [151]:
trainer.train()

  0%|          | 0/7750 [00:00<?, ?it/s]

{'loss': 2.5837, 'learning_rate': 9.622068965517242e-05, 'epoch': 1.0}


  0%|          | 0/86 [00:00<?, ?it/s]

{'eval_loss': 2.2370874881744385, 'eval_runtime': 3.0156, 'eval_samples_per_second': 912.261, 'eval_steps_per_second': 28.519, 'epoch': 1.0}
{'loss': 2.2602, 'learning_rate': 8.55448275862069e-05, 'epoch': 2.0}


  0%|          | 0/86 [00:00<?, ?it/s]

...

{'eval_loss': 1.947245717048645, 'eval_runtime': 3.0533, 'eval_samples_per_second': 900.991, 'eval_steps_per_second': 28.166, 'epoch': 9.0}
{'loss': 1.9985, 'learning_rate': 1.5172413793103449e-07, 'epoch': 9.99}


  0%|          | 0/86 [00:00<?, ?it/s]

Overwriting existing adapter 'inv_source_mlm'.


{'eval_loss': 1.9326860904693604, 'eval_runtime': 3.0308, 'eval_samples_per_second': 907.668, 'eval_steps_per_second': 28.375, 'epoch': 10.0}
{'train_runtime': 547.6569, 'train_samples_per_second': 452.765, 'train_steps_per_second': 14.151, 'train_loss': 2.1359461369668282, 'epoch': 10.0}


TrainOutput(global_step=7750, training_loss=2.1359461369668282, metrics={'train_runtime': 547.6569, 'train_samples_per_second': 452.765, 'train_steps_per_second': 14.151, 'train_loss': 2.1359461369668282, 'epoch': 10.0})

In [152]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/86 [00:00<?, ?it/s]

>>> Perplexity: 7.03


In [153]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [1]:
import torch
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import pipeline

mask_filler = pipeline("fill-mask", model=trainer.model, tokenizer=tokenizer)

NameError: name 'model' is not defined

In [158]:
masked_sentence = "I bought a [MASK]."
preds = mask_filler(masked_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> i cancelled my job.
>>> i cancelled my vacation.
>>> i cancelled my plans.
>>> i cancelled my trip.
>>> i cancelled my flight.
